# Applicative Exercises

## IGNORE (Configuration and Dependencies)

In [1]:
:set -XNoImplicitPrelude
:set -XTypeSynonymInstances
:set -XInstanceSigs
:load ../../original-source/Course/Core.hs
:load ../../original-source/Course/ExactlyOne.hs
:load ../../original-source/Course/Optional.hs
:load ../../original-source/Course/List.hs
:load ../../original-source/Course/Functor.hs
import qualified Prelude as P
import Data.Foldable (for_, traverse_)
import Test.QuickCheck

runTests :: [(P.String, Property)] -> IO ()
runTests tests = for_ tests $ \(name, prop) -> do
  P.putStrLn $ "Testing: " P.++ name
  quickCheck prop

## The Applicative Typeclass

### Definition

In [2]:
class (Functor f) => Applicative f where
-- mempty   :: a
  pure :: a -> f a
  -- an alpha numeric name breaks the elegance of the usage
  -- of this (<*>) operator, please wait and see
  -- But we refer to it as "Apply"
-- mappend :: a          -> a   -> a
  (<*>)    :: f (a -> b) -> f a -> f b

infixl 4 <*>

**Discuss what the `(Functor f) =>` part means**

### Laws

All instances of the `Applicative` type class must satisfy the following laws

The laws can look quite baffling when you see them even in their "simplest" form,
Each one has a correspondence to a `Monoid` law however, which may be easier to
grok, I will supply the `Monoid` law in each case so that you can hopefully get a
qualitative understanding of it.

#### The law of left Identity

`pure f <*>` is *equivalent* to `fmap f` which as we learnt in the previous module,
**does nothing to the structure, it just applies `f` inside**.

You might have wondered what `pure` is or does.

*This defines pure*, in terms of `<*>`.

```
∀x. pure f <*> x ≅ f <$> x
```

Just as `mempty` in `Monoid` is *defined* to be the value such that `mappend mempty` *does nothing* to its
input, `pure` is similarly defined.

This corresponds to the *left* part of the *Identity Law* for Monoids:

```
∀x. mempty <> x ≅ x
```

#### The law of right Identity

`<*> pure x` is *equivalent* to sending x inside the structure and applying the function inside to it.

```
∀mf x. mf <*> pure x ≅ fmap (\f -> f x) mf
```

This corresponds to the *right* part of the *Identity Law* for Monoids.

```
∀x. x <> mempty ≅ x
```

#### The law of associativity

```
∀u v w. pure (.) <*> u <*> v <*> w ≅ u <*> (v <*> w)
```

**Discuss this one amongst the group (Now or leave till later).**

This corresponds to the *associative law* for Monoids:

```
(x <> y) <> z ≅ x <> (y <> z)
```

### Motivation

Understanding of the problem this typeclass solves is left until *after* the exercises, please be patient and save deep questions for later.

Suffice it to say, the `Functor` abstraction only gets us so far, `Applicative` picks up the baton and carries us through the next level.

### Example Instance

In [3]:
instance Applicative ExactlyOne where
  pure :: a -> ExactlyOne a
  pure = ExactlyOne
  
  (<*>) :: ExactlyOne (a -> b) -> ExactlyOne a -> ExactlyOne b
  (ExactlyOne f) <*> (ExactlyOne x) = ExactlyOne (f x)

Play around with this a bit:

In [4]:
add3 :: Int -> Int -> Int -> Int
add3 x y z = x + y + z

add3 2 4 6

12

In [5]:
add3 <$> ExactlyOne 2 <*> ExactlyOne 4 <*> ExactlyOne 6

ExactlyOne 12

Break this up a bit:

In [6]:
appExample = add3 <$> ExactlyOne 2
:t add3 2
:t appExample

add3 2 :: Int -> Int -> Int

appExample :: ExactlyOne (Int -> Int -> Int)

In [7]:
appExample2 = add3 <$> ExactlyOne 2 <*> ExactlyOne 4
:t add3 2 4
:t appExample2

add3 2 4 :: Int -> Int

appExample2 :: ExactlyOne (Int -> Int)

In [8]:
appExample = add3 <$> ExactlyOne 2 <*> ExactlyOne 4 <*> ExactlyOne 6
:t add3 2 4 6
:t appExample
appExample

add3 2 4 6 :: Int

appExample :: ExactlyOne Int

ExactlyOne 12

## Exercises

### 1. Write a `Applicative` instance for `List`

There is a valid instance of `Applicative` for the `List` data type.

Implement `pure` and `<*>` for `List`

#### Exercise

In [15]:
instance Applicative List where
  pure :: a -> List a
  pure = _pure
  
  (<*>) :: List (a -> b) -> List a -> List b
  (<*>) = _apply
  

#### Examples

In [16]:
-- >>> (*) <$> 1 :. 2 :. Nil <*> 4 :. 5 :. 6 :. Nil
-- [4,5,6,8,10,12]
(*) <$> 1 :. 2 :. Nil <*> 4 :. 5 :. 6 :. Nil

[4,5,6,8,10,12]

In [17]:
-- >>> (+1) :. (*2) :. Nil <*> 1 :. 2 :. 3 :. Nil
-- [2,3,4,2,4,6]
(+1) :. (*2) :. Nil <*> 1 :. 2 :. 3 :. Nil

[2,3,4,2,4,6]

#### Tests (Do not modify these, just run them)

In [18]:
-- prop> pure x == x :. Nil
listPureTest :: Property
listPureTest = forAll (arbitrary :: Gen Int) $ \int -> pure int === int :. Nil

-- >>> (+1) :. (*2) :. Nil <*> 1 :. 2 :. 3 :. Nil
-- [2,3,4,2,4,6]
listApTest :: Property
listApTest =
  let
    lhs = (+1) :. (*2) :. Nil <*> 1 :. 2 :. 3 :. Nil
    rhs = 2 :. 3 :. 4 :. 2 :. 4 :. 6 :. Nil
  in once $ lhs === rhs

testList :: IO ()
testList = runTests [
    ("pure x == x :. Nil", listPureTest)
  , ("(+1) :. (*2) :. Nil <*> 1 :. 2 :. 3 :. Nil == 2 :. 3 :. 4 :. 2 :. 4 :. 6 :. Nil", listApTest)
  ]

testList

Testing: pure x == x :. Nil
+++ OK, passed 100 tests.
Testing: (+1) :. (*2) :. Nil <*> 1 :. 2 :. 3 :. Nil == 2 :. 3 :. 4 :. 2 :. 4 :. 6 :. Nil
+++ OK, passed 1 tests.

### 2. Work backwards and derive `<$>` from `Applicative`

Even if we didn't have the `Functor f` constraint in the definition of `Applicative`,
`pure`, `<*>` and the laws imply the existence of `fmap / <$>`.

Prove this for yourself. Implement `fmap` below (we will call it `<$$>` this time) without calling `fmap` or `<$>`.

#### Exercise

In [19]:
(<$$>) :: (Applicative f) => (a -> b) -> f a -> f b
(<$$>) = _fmap

#### Play around with it

In [20]:
-- >>> (+1) <$$> Nil
-- Nil
(+1) <$$> Nil

[]

In [21]:
-- >>> (+1) <$$> (1 :. 2 :. 3 :. Nil)
(+1) <$$> (1 :. 2 :. 3 :. Nil)

[2,3,4]

#### Tests (Do not modify these, just run them)

In [22]:
apFmapTest1 :: Property
apFmapTest1 =
  let
    lhs = (+1) <$$> (ExactlyOne 2)
    
    rhs :: ExactlyOne Int
    rhs = ExactlyOne 3
  in once $ lhs === rhs

apFmapTest2 :: Property
apFmapTest2 = once $
  ((+1) <$$> Nil) === Nil

apFmapTest3 :: Property
apFmapTest3 = once $
  (+ 1) <$$> (1 :. 2 :. 3 :. Nil) === (2 :. 3 :. 4 :. Nil)

apFmapTests :: IO ()
apFmapTests = runTests [
    ("(+1) <$$> (ExactlyOne 2) === ExactlyOne 3", apFmapTest1)
  , ("((+1) <$$> Nil) === Nil", apFmapTest2)
  , ("(+ 1) <$$> (1 :. 2 :. 3 :. Nil) === (2 :. 3 :. 4 :. Nil)", apFmapTest3)
  ]

apFmapTests

Testing: (+1) <$$> (ExactlyOne 2) === ExactlyOne 3
+++ OK, passed 1 tests.
Testing: ((+1) <$$> Nil) === Nil
+++ OK, passed 1 tests.
Testing: (+ 1) <$$> (1 :. 2 :. 3 :. Nil) === (2 :. 3 :. 4 :. Nil)
+++ OK, passed 1 tests.

### 3. Write `Applicative` instance for `Optional`

#### Exercise

In [24]:
instance Applicative Optional where
  pure :: a -> Optional a
  pure = _pure
  
  (<*>) :: Optional (a -> b) -> Optional a -> Optional b
  (<*>) = _apply

#### Play around with it

In [25]:
-- >>> Full (+8) <*> Full 7
-- Full 15
Full (+8) <*> Full 7

Full 15

In [26]:
-- >>> Empty <*> Full 7
-- Empty
Empty <*> Full 7

Empty

#### Tests (Do not modify these, just run them)

In [27]:
optionalPureTest :: Property
optionalPureTest =
  forAll (arbitrary :: Gen Int) $ \int ->
    pure int === Full int

optionalFullFull :: Property
optionalFullFull = once $
  (Full (+8) <*> Full 7) === Full 15

optionalEmptyFull :: Property
optionalEmptyFull = forAll (arbitrary :: Gen P.String) $ \string ->
  (Empty <*> Full string) === (Empty :: Optional Int)

optionalFullEmpty :: Property
optionalFullEmpty = forAll (arbitrary :: Gen Int) $ \int ->
  (Full (+ int) <*> Empty) === Empty


optionalTests :: IO ()
optionalTests = runTests [
    ("pure int === Full int", optionalPureTest)
  , ("Full (+8) <*> Full 7 === Full 15", optionalFullFull)
  , ("(Empty <*> Full string) === Empty", optionalEmptyFull)
  , ("(Full (+ int) <*> Empty) === Empty", optionalFullEmpty)
  ]

optionalTests

Testing: pure int === Full int
+++ OK, passed 100 tests.
Testing: Full (+8) <*> Full 7 === Full 15
+++ OK, passed 1 tests.
Testing: (Empty <*> Full string) === Empty
+++ OK, passed 100 tests.
Testing: (Full (+ int) <*> Empty) === Empty
+++ OK, passed 100 tests.

### 4. Lift a binary function

`fmap` "lifts" a unary function, its type signature is:

``` haskell
fmap :: (Functor f) => (a -> b) -> f a -> f b
```

If we give `fmap` a binary function then the output type `b`
is a unary function, i.e `b ~ c -> d` and type looks like:

``` haskell
fmap :: (Functor f) => (a -> c -> d) -> f a -> f (c -> d)
```

But we want:

``` haskell
lift2 :: (Applicative f) => (a -> c -> d) -> f a -> f c -> f d
```

#### Exercise

In [28]:
lift2 :: (Applicative f) => (a -> b -> c) -> f a -> f b -> f c
lift2 = _lift2

#### Play around with it

In [29]:
-- >>> lift2 (+) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil)
-- [5,6,6,7,7,8]
lift2 (+) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil)

[5,6,6,7,7,8]

In [30]:
-- >>> lift2 (+) (Full 7) (Full 8)
-- Full 15
lift2 (+) (Full 7) (Full 8)

Full 15

In [31]:
-- >>> lift2 (+) (Full 7) Empty
-- Empty
lift2 (+) (Full 7) Empty

Empty

In [32]:
-- >>> lift2 (+) Empty (Full 8)
-- Empty
lift2 (+) Empty (Full 8)

Empty

#### Tests (Do not modify these, just run them)

In [33]:
lift2Test1 :: Property
lift2Test1 =
  let
    lhs :: List Int
    lhs = lift2 (+) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil)
    rhs = 5 :. 6 :. 6 :. 7 :. 7 :. 8 :. Nil
  in once $ lhs === rhs

lift2Test2 :: Property
lift2Test2 = once $
  lift2 (+) (Full 7) (Full 8) === Full 15

lift2Test3 :: Property
lift2Test3 = once $
  lift2 (+) (Full 7) Empty === Empty

runLift2Tests :: IO ()
runLift2Tests = runTests [
    ("lift2 (+) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) === 5 :. 6 :. 6 :. 7 :. 7 :. 8 :. Nil", lift2Test1)
  , ("lift2 (+) (Full 7) (Full 8) === Full 15", lift2Test2)
  , ("lift2 (+) (Full 7) Empty === Empty", lift2Test3)
  ]

runLift2Tests

Testing: lift2 (+) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) === 5 :. 6 :. 6 :. 7 :. 7 :. 8 :. Nil
+++ OK, passed 1 tests.
Testing: lift2 (+) (Full 7) (Full 8) === Full 15
+++ OK, passed 1 tests.
Testing: lift2 (+) (Full 7) Empty === Empty
+++ OK, passed 1 tests.

### Lift a ternary function

#### Exercise

In [34]:
lift3 :: (Applicative f) => (a -> b -> c -> d) -> f a -> f b -> f c -> f d
lift3 = _lift3

#### Play around with it

In [35]:
-- >>> lift3 (\a b c -> a + b + c) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9)
-- ExactlyOne 24
lift3 (\a b c -> a + b + c) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9)

ExactlyOne 24

In [36]:
-- >>> lift3 (\a b c -> a + b + c) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil)
-- [11,12,13,12,13,14,12,13,14,13,14,15,13,14,15,14,15,16]
lift3 (\a b c -> a + b + c) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil)

[11,12,13,12,13,14,12,13,14,13,14,15,13,14,15,14,15,16]

In [37]:
-- >>> lift3 (\a b c -> a + b + c) (Full 7) (Full 8) (Full 9)
-- Full 24
lift3 (\a b c -> a + b + c) (Full 7) (Full 8) (Full 9)

Full 24

In [38]:
-- >>> lift3 (\a b c -> a + b + c) (Full 7) (Full 8) Empty
-- Empty
lift3 (\a b c -> a + b + c) (Full 7) (Full 8) Empty

Empty

In [39]:
-- >>> lift3 (\a b c -> a + b + c) Empty (Full 8) (Full 9)
-- Empty
lift3 (\a b c -> a + b + c) Empty (Full 8) (Full 9)

Empty

In [40]:
-- >>> lift3 (\a b c -> a + b + c) Empty Empty (Full 9)
-- Empty
lift3 (\a b c -> a + b + c) Empty Empty (Full 9)

Empty

#### Tests

In [41]:
lift3Test1 :: Property
lift3Test1 = once $
  lift3 (\a b c -> a + b + c) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) === ExactlyOne 24

lift3Test2 :: Property
lift3Test2 =
  let
    lhs :: List Int
    lhs = lift3 (\a b c -> a + b + c) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil)
    rhs = P.foldr (:.) Nil [11, 12, 13, 12, 13, 14, 12, 13, 14, 13, 14, 15, 13, 14, 15, 14, 15, 16]
  in once $ lhs === rhs

lift3Test3 :: Property
lift3Test3 = once $
  lift3 (\a b c -> a + b + c) (Full 7) (Full 8) (Full 9) === Full 24

lift3Test4 :: Property
lift3Test4 = once $
  lift3 (\a b c -> a + b + c) (Full 7) (Full 8) Empty === Empty

lift3Test5 :: Property
lift3Test5 = once $
  lift3 (\a b c -> a + b + c) Empty (Full 8) (Full 9) === Empty

lift3Test6 :: Property
lift3Test6 = once $
  lift3 (\a b c -> a + b + c) Empty Empty (Full 9) === Empty

runLift3Tests :: IO ()
runLift3Tests = runTests [
    ("lift3 (\a b c -> a + b + c) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) === ExactlyOne 24", lift3Test1)
  , ("lift3 (\a b c -> a + b + c) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) === [11, 12, 13, 12, 13, 14, 12, 13, 14, 13, 14, 15, 13, 14, 15, 14, 15, 16]", lift3Test2)
  , ("lift3 (\a b c -> a + b + c) (Full 7) (Full 8) (Full 9) === Full 24", lift3Test3)
  , ("lift3 (\a b c -> a + b + c) (Full 7) (Full 8) Empty === Empty", lift3Test4)
  , ("lift3 (\a b c -> a + b + c) Empty (Full 8) (Full 9) === Empty", lift3Test5)
  , ("lift3 (\a b c -> a + b + c) Empty Empty (Full 9) === Empty", lift3Test6)
  ]

runLift3Tests

Testing: lift3 ( b c -> a + b + c) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) === ExactlyOne 24
+++ OK, passed 1 tests.
Testing: lift3 ( b c -> a + b + c) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) === [11, 12, 13, 12, 13, 14, 12, 13, 14, 13, 14, 15, 13, 14, 15, 14, 15, 16]
+++ OK, passed 1 tests.
Testing: lift3 ( b c -> a + b + c) (Full 7) (Full 8) (Full 9) === Full 24
+++ OK, passed 1 tests.
Testing: lift3 ( b c -> a + b + c) (Full 7) (Full 8) Empty === Empty
+++ OK, passed 1 tests.
Testing: lift3 ( b c -> a + b + c) Empty (Full 8) (Full 9) === Empty
+++ OK, passed 1 tests.
Testing: lift3 ( b c -> a + b + c) Empty Empty (Full 9) === Empty
+++ OK, passed 1 tests.

### 5. Lift a quarternary function

#### Exercise

In [42]:
lift4
  :: (Applicative f)
  => (a -> b -> c -> d -> e)
  -> f a -> f b -> f c -> f d -> f e
lift4 = _lift4

#### Try it out

In [43]:
-- >>> lift4 (\a b c d -> a + b + c + d) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) (ExactlyOne 10)
-- ExactlyOne 34
lift4 (\a b c d -> a + b + c + d) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) (ExactlyOne 10)

ExactlyOne 34

In [44]:
-- >>> lift4 (\a b c d -> a + b + c + d) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) (9 :. 10 :. Nil)
-- [20,21,21,22,22,23,21,22,22,23,23,24,21,22,22,23,23,24,22,23,23,24,24,25,22,23,23,24,24,25,23,24,24,25,25,26]
lift4 (\a b c d -> a + b + c + d) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) (9 :. 10 :. Nil)

[20,21,21,22,22,23,21,22,22,23,23,24,21,22,22,23,23,24,22,23,23,24,24,25,22,23,23,24,24,25,23,24,24,25,25,26]

In [45]:
-- >>> lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) (Full 9) (Full 10)
-- Full 34
lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) (Full 9) (Full 10)

Full 34

In [46]:
-- >>> lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) Empty  (Full 10)
-- Empty
lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) Empty  (Full 10)

Empty

In [47]:
-- >>> lift4 (\a b c d -> a + b + c + d) Empty (Full 8) (Full 9) (Full 10)
-- Empty
lift4 (\a b c d -> a + b + c + d) Empty (Full 8) (Full 9) (Full 10)

Empty

In [48]:
-- >>> lift4 (\a b c d -> a + b + c + d) Empty Empty (Full 9) (Full 10)
-- Empty
lift4 (\a b c d -> a + b + c + d) Empty Empty (Full 9) (Full 10)

Empty

#### Tests

In [49]:
lift4Test1 :: Property
lift4Test1 = once $
  lift4 (\a b c d -> a + b + c + d) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) (ExactlyOne 10) === ExactlyOne 34

lift4Test2 :: Property
lift4Test2 =
  let
    lhs :: List Int
    lhs = lift4 (\a b c d -> a + b + c + d) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) (9 :. 10 :. Nil)
    rhs = P.foldr (:.) Nil [20,21,21,22,22,23,21,22,22,23,23,24,21,22,22,23,23,24,22,23,23,24,24,25,22,23,23,24,24,25,23,24,24,25,25,26]
  in once $ lhs === rhs

lift4Test3 :: Property
lift4Test3 = once $
  lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) (Full 9) (Full 10) === Full 34

lift4Test4 :: Property
lift4Test4 = once $
  lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) Empty  (Full 10) === Empty

lift4Test5 :: Property
lift4Test5 = once $
  lift4 (\a b c d -> a + b + c + d) Empty (Full 8) (Full 9) (Full 10) === Empty

lift4Test6 :: Property
lift4Test6 = once $
  lift4 (\a b c d -> a + b + c + d) Empty Empty (Full 9) (Full 10) === Empty

lift4Tests :: IO ()
lift4Tests = runTests [
    ("lift4 (\a b c d -> a + b + c + d) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) (ExactlyOne 10) === ExactlyOne 34", lift4Test1)
  , ("lift4 (\a b c d -> a + b + c + d) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) (9 :. 10 :. Nil)", lift4Test2)
  , ("lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) (Full 9) (Full 10) === Full 34", lift4Test3)
  , ("lift4 (\a b c d -> a + b + c + d) (Full 7) (Full 8) Empty  (Full 10) === Empty", lift4Test4)
  , ("lift4 (\a b c d -> a + b + c + d) Empty (Full 8) (Full 9) (Full 10) == Empty", lift4Test5)
  , ("lift4 (\a b c d -> a + b + c + d) Empty Empty (Full 9) (Full 10) === Empty", lift4Test6)
  ]

lift4Tests

Testing: lift4 ( b c d -> a + b + c + d) (ExactlyOne 7) (ExactlyOne 8) (ExactlyOne 9) (ExactlyOne 10) === ExactlyOne 34
+++ OK, passed 1 tests.
Testing: lift4 ( b c d -> a + b + c + d) (1 :. 2 :. 3 :. Nil) (4 :. 5 :. Nil) (6 :. 7 :. 8 :. Nil) (9 :. 10 :. Nil)
+++ OK, passed 1 tests.
Testing: lift4 ( b c d -> a + b + c + d) (Full 7) (Full 8) (Full 9) (Full 10) === Full 34
+++ OK, passed 1 tests.
Testing: lift4 ( b c d -> a + b + c + d) (Full 7) (Full 8) Empty  (Full 10) === Empty
+++ OK, passed 1 tests.
Testing: lift4 ( b c d -> a + b + c + d) Empty (Full 8) (Full 9) (Full 10) == Empty
+++ OK, passed 1 tests.
Testing: lift4 ( b c d -> a + b + c + d) Empty Empty (Full 9) (Full 10) === Empty
+++ OK, passed 1 tests.

### 6. Apply, discarding the value of the first argument

Apply, discarding the value of the first argument.

Pronounced, "right apply".

#### Exercise

In [51]:
(*>) :: (Applicative f) => f a -> f b -> f b
(*>) = _rightApply

#### Play around with it

In [52]:
-- >>> (1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. 6 :. Nil)
-- [4,5,6,4,5,6,4,5,6]
(1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. 6 :. Nil)

[4,5,6,4,5,6,4,5,6]

In [53]:
-- >>> (1 :. 2 :. Nil) *> (4 :. 5 :. 6 :. Nil)
-- [4,5,6,4,5,6]
(1 :. 2 :. Nil) *> (4 :. 5 :. 6 :. Nil)

[4,5,6,4,5,6]

In [54]:
-- >>> (1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. Nil)
-- [4,5,4,5,4,5]
(1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. Nil)

[4,5,4,5,4,5]

In [55]:
-- >>> Full 7 *> Full 8
-- Full 8
Full 7 *> Full 8

Full 8

#### Tests

In [56]:
rightApplyTest1 :: Property
rightApplyTest1 =
  let
    lhs = (1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. 6 :. Nil)
    rhs = P.foldr (:.) Nil [4,5,6,4,5,6,4,5,6]
  in once $ lhs === rhs

rightApplyTest2 :: Property
rightApplyTest2 = once $
  (1 :. 2 :. Nil) *> (4 :. 5 :. 6 :. Nil) === (4 :. 5 :. 6 :. 4 :. 5 :. 6 :. Nil)

rightApplyTest3 :: Property
rightApplyTest3 = once $
  (1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. Nil) === (4 :. 5 :. 4 :. 5 :. 4 :. 5 :. Nil)

rightApplyTest4 :: Property
rightApplyTest4 = once $
  Full 7 *> Full 8 === Full 8

rightApplyList :: Property
rightApplyList =
  let
    gen :: Gen (Int, Int, Int, Int, Int, Int)
    gen = do
      a <- arbitrary
      b <- arbitrary
      c <- arbitrary
      x <- arbitrary
      y <- arbitrary
      z <- arbitrary
      P.return (a, b, c, x, y, z)
  in forAll gen $ \(a, b, c, x, y, z) ->
    (a :. b :. c :. Nil) *> (x :. y :. z :. Nil) === (x :. y :. z :. x :. y :. z :. x :. y :. z :. Nil)
    
rightApplyOptional :: Property
rightApplyOptional =
  let
    gen :: Gen (Int, Int)
    gen = do
      x <- arbitrary
      y <- arbitrary
      P.return (x, y)
  in forAll gen $ \(x, y) ->
    Full x *> Full y === Full y

runRightApplyTests :: IO ()
runRightApplyTests = runTests [
    ("(1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. 6 :. Nil)", rightApplyTest1)
  , ("(1 :. 2 :. Nil) *> (4 :. 5 :. 6 :. Nil) === (4 :. 5 :. 6 :. 4 :. 5 :. 6 :. Nil)", rightApplyTest2)
  , ("(1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. Nil) === (4 :. 5 :. 4 :. 5 :. 4 :. 5 :. Nil)", rightApplyTest3)
  , ("Full 7 *> Full 8 === Full 8", rightApplyTest4)
  , ("*> over List", rightApplyList)
  , ("*> over Optional", rightApplyOptional)
  ]

runRightApplyTests

Testing: (1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. 6 :. Nil)
+++ OK, passed 1 tests.
Testing: (1 :. 2 :. Nil) *> (4 :. 5 :. 6 :. Nil) === (4 :. 5 :. 6 :. 4 :. 5 :. 6 :. Nil)
+++ OK, passed 1 tests.
Testing: (1 :. 2 :. 3 :. Nil) *> (4 :. 5 :. Nil) === (4 :. 5 :. 4 :. 5 :. 4 :. 5 :. Nil)
+++ OK, passed 1 tests.
Testing: Full 7 *> Full 8 === Full 8
+++ OK, passed 1 tests.
Testing: *> over List
+++ OK, passed 100 tests.
Testing: *> over Optional
+++ OK, passed 100 tests.

### 7. Apply, discarding the value of the second argument

Referred to as "left apply"

#### Exercise

In [57]:
(<*) :: (Applicative f) => f a -> f b -> f a
(<*) = _leftApply

#### Try it out

In [58]:
-- >>> (1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. 6 :. Nil)
-- [1,1,1,2,2,2,3,3,3]
(1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. 6 :. Nil)

[1,1,1,2,2,2,3,3,3]

In [59]:
-- >>> (1 :. 2 :. Nil) <* (4 :. 5 :. 6 :. Nil)
-- [1,1,1,2,2,2]
(1 :. 2 :. Nil) <* (4 :. 5 :. 6 :. Nil)

[1,1,1,2,2,2]

In [60]:
-- >>> (1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. Nil)
-- [1,1,2,2,3,3]
(1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. Nil)

[1,1,2,2,3,3]

In [61]:
-- >>> Full 7 <* Full 8
-- Full 7
Full 7 <* Full 8

Full 7

#### Tests

In [62]:
runLeftApplyTest1 :: Property
runLeftApplyTest1 =
  let
    lhs = (1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. 6 :. Nil)
    rhs = P.foldr (:.) Nil [1,1,1,2,2,2,3,3,3]
  in once $ lhs === rhs

runLeftApplyTest2 :: Property
runLeftApplyTest2 =
  let
    lhs = (1 :. 2 :. Nil) <* (4 :. 5 :. 6 :. Nil)
    rhs = P.foldr (:.) Nil [1,1,1,2,2,2]
  in once $ lhs === rhs
  
runLeftApplyTest3 :: Property
runLeftApplyTest3 =
  let
    lhs = (1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. Nil)
    rhs = P.foldr (:.) Nil [1,1,2,2,3,3]
  in once $ lhs === rhs

runLeftApplyTest4 :: Property
runLeftApplyTest4 = once $
  (Full 7 <* Full 8) === Full 7

runLeftApplyList :: Property
runLeftApplyList =
  let
    gen :: Gen (Int, Int, Int, Int, Int, Int)
    gen = do
      a <- arbitrary
      b <- arbitrary
      c <- arbitrary
      x <- arbitrary
      y <- arbitrary
      z <- arbitrary
      P.return (a, b, c, x, y, z)
  in forAll gen $ \(a, b, c, x, y, z) ->
    ((x :. y :. z :. Nil) <* (a :. b :. c :. Nil)) === (x :. x :. x :. y :. y :. y :. z :. z :. z :. Nil)

runLeftApplyOptional :: Property
runLeftApplyOptional =
  let
    gen :: Gen (Int, Int)
    gen = do
      x <- arbitrary
      y <- arbitrary
      P.return (x, y)
  in forAll gen $ \(x, y) ->
    (Full x <* Full y) === Full x

runLeftApplyTests :: IO ()
runLeftApplyTests = runTests [
    ("(1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. 6 :. Nil) == ?", runLeftApplyTest1)
  , ("(1 :. 2 :. Nil) <* (4 :. 5 :. 6 :. Nil) == ?", runLeftApplyTest2)
  , ("(1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. Nil) == ?", runLeftApplyTest3)
  , ("(Full 7 <* Full 8) === Full 7", runLeftApplyTest4)
  , ("<* over Lists", runLeftApplyList)
  , ("<* over Optionals", runLeftApplyOptional)
  ]

runLeftApplyTests

Testing: (1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. 6 :. Nil) == ?
+++ OK, passed 1 tests.
Testing: (1 :. 2 :. Nil) <* (4 :. 5 :. 6 :. Nil) == ?
+++ OK, passed 1 tests.
Testing: (1 :. 2 :. 3 :. Nil) <* (4 :. 5 :. Nil) == ?
+++ OK, passed 1 tests.
Testing: (Full 7 <* Full 8) === Full 7
+++ OK, passed 1 tests.
Testing: <* over Lists
+++ OK, passed 100 tests.
Testing: <* over Optionals
+++ OK, passed 100 tests.

### 8. `sequence`

> Sequences a list of structures to a structure of list

Or:

> [Transposes](https://encrypted.google.com/search?hl=en&q=define%3A%20transpose) `List` and `f`.

#### Exercises

In [68]:
sequence :: (Applicative f) => List (f a) -> f (List a)
sequence = _sequence

#### Try it out

In [69]:
-- >>> sequence (ExactlyOne 7 :. ExactlyOne 8 :. ExactlyOne 9 :. Nil)
-- ExactlyOne [7,8,9]
sequence (ExactlyOne 7 :. ExactlyOne 8 :. ExactlyOne 9 :. Nil)

ExactlyOne [7,8,9]

In [70]:
-- >>> sequence ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)
-- [[1,1],[1,2],[2,1],[2,2],[3,1],[3,2]]
sequence ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)

[[1,1],[1,2],[2,1],[2,2],[3,1],[3,2]]

In [71]:
-- >>> sequence (Full 7 :. Empty :. Nil)
-- Empty
sequence (Full 7 :. Empty :. Nil)

Empty

In [72]:
-- >>> sequence (Full 7 :. Full 8 :. Nil)
-- Full [7,8]
sequence (Full 7 :. Full 8 :. Nil)

Full [7,8]

#### Tests

In [73]:
sequenceExactlyOne :: Property
sequenceExactlyOne =
  let
    lhs :: ExactlyOne (List Int)
    lhs = sequence (ExactlyOne 7 :. ExactlyOne 8 :. ExactlyOne 9 :. Nil)
    rhs = ExactlyOne $ 7 :. 8 :. 9 :. Nil
  in once $ lhs === rhs

sequenceList :: Property
sequenceList =
  let
    lhs :: List (List Int)
    lhs = sequence ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)
    rhs = fmap (P.foldr (:.) Nil) . P.foldr (:.) Nil $ [[1,1],[1,2],[2,1],[2,2],[3,1],[3,2]]
  in once $ lhs === rhs

sequenceOptional1 :: Property
sequenceOptional1 = once $
  sequence (Full 7 :. Empty :. Nil) === Empty

sequenceOptional2 :: Property
sequenceOptional2 = once $
  sequence (Full 7 :. Full 8 :. Nil) === Full (7 :. 8 :. Nil)
  

runSequenceTests :: IO ()
runSequenceTests = runTests [
    ("Sequence ExactlyOne", sequenceExactlyOne)
  , ("Sequence Lists", sequenceList)
  , ("Sequence a List of Optionals with an Empty in it", sequenceOptional1)
  , ("Sequence a List of Optionals with no Emptys in it", sequenceOptional2)
  ]

runSequenceTests

Testing: Sequence ExactlyOne
+++ OK, passed 1 tests.
Testing: Sequence Lists
+++ OK, passed 1 tests.
Testing: Sequence a List of Optionals with an Empty in it
+++ OK, passed 1 tests.
Testing: Sequence a List of Optionals with no Emptys in it
+++ OK, passed 1 tests.

### 9. `replicateA`

Replicate an effect a given number of times.

#### Exercise

In [74]:
replicateA :: (Applicative f) => Int -> f a -> f (List a)
replicateA = _replicateA

#### Try it out

In [75]:
-- >>> replicateA 4 (ExactlyOne "hi")
-- ExactlyOne ["hi","hi","hi","hi"]
replicateA 4 (ExactlyOne "hi")

ExactlyOne ["hi","hi","hi","hi"]

In [76]:
-- >>> replicateA 4 (Full "hi")
-- Full ["hi","hi","hi","hi"]
replicateA 4 (Full "hi")

Full ["hi","hi","hi","hi"]

In [77]:
-- >>> replicateA 4 Empty
-- Empty
replicateA 4 Empty

Empty

In [78]:
-- >>> replicateA 3 ('a' :. 'b' :. 'c' :. Nil)
-- ["aaa","aab","aac","aba","abb","abc","aca","acb","acc","baa","bab","bac","bba","bbb","bbc","bca","bcb","bcc","caa","cab","cac","cba","cbb","cbc","cca","ccb","ccc"]
replicateA 3 ('a' :. 'b' :. 'c' :. Nil)

["aaa","aab","aac","aba","abb","abc","aca","acb","acc","baa","bab","bac","bba","bbb","bbc","bca","bcb","bcc","caa","cab","cac","cba","cbb","cbc","cca","ccb","ccc"]

#### Tests

In [80]:
replicateExactlyOne :: Property
replicateExactlyOne = once $
  replicateA 4 (ExactlyOne "hi") === ExactlyOne ("hi" :. "hi" :. "hi" :. "hi" :. Nil)

replicateFullOptionalLength :: Property
replicateFullOptionalLength = forAll (arbitrary :: Gen (NonNegative Int)) $ \(NonNegative length') ->
  (length <$> replicateA length' (Full "hi")) === Full length'

replicateFullOptionalContents :: Property
replicateFullOptionalContents =
  let
    gen :: Gen (Int, Int)
    gen = do
      length' <- (arbitrary :: Gen (NonNegative Int))
      content <- (arbitrary :: Gen Int)
      case length' of
        NonNegative x -> P.return (x, content)
  in forAll gen $ \(length', content) ->
    replicateA length' (Full content) === Full (replicate length' content)

replicateEmptyOptional :: Property
replicateEmptyOptional = forAll (arbitrary :: Gen (Positive Int)) $ \(Positive n) ->
  replicateA n Empty === Empty

zeroReplicateOptional :: Property
zeroReplicateOptional = once $
  replicateA 0 Empty === Full Nil

runReplicateTests :: IO ()
runReplicateTests = runTests [
    ("Replicate ExactlyOne", replicateExactlyOne)
  , ("The length of the returned list should be the same as the length given to replicateA", replicateFullOptionalLength)
  , ("replicateA should be getting its content from the structure/action it received", replicateFullOptionalContents)
  , ("For all n > 0, replicateA n Empty === Empty", replicateEmptyOptional)
  , ("replicateA 0 Empty === Full []", zeroReplicateOptional)
  ]

runReplicateTests

Testing: Replicate ExactlyOne
+++ OK, passed 1 tests.
Testing: The length of the returned list should be the same as the length given to replicateA
+++ OK, passed 100 tests.
Testing: replicateA should be getting its content from the structure/action it received
+++ OK, passed 100 tests.
Testing: For all n > 0, replicateA n Empty === Empty
+++ OK, passed 100 tests.
Testing: replicateA 0 Empty === Full []
+++ OK, passed 1 tests.

### 10. `filtering`

Filter a list with a predicate that produces an effect.

#### Exercise

In [84]:
filtering :: (Applicative f) => (a -> f Bool) -> List a -> f (List a)
filtering = _filtering

#### Try it out

In [85]:
-- >>> filtering (ExactlyOne . even) (4 :. 5 :. 6 :. Nil)
-- ExactlyOne [4,6]
filtering (ExactlyOne . even) (4 :. 5 :. 6 :. Nil)

ExactlyOne [4,6]

In [86]:
-- >>> filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. Nil)
-- Full [4,5,6]
filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. Nil)

Full [4,5,6]

In [87]:
-- >>> filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 7 :. 8 :. 9 :. Nil)
-- Full [4,5,6,7]
filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 7 :. 8 :. 9 :. Nil)

Full [4,5,6,7]

In [88]:
-- >>> filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 13 :. 14 :. Nil)
-- Empty
filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 13 :. 14 :. Nil)

Empty

In [115]:
-- >>> filtering (const $ True :. True :.  Nil) (1 :. 2 :. 3 :. Nil)
-- [[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3]]
filtering (const $ True :. True :.  Nil) (1 :. 2 :. 3 :. Nil)

[[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3]]

#### Tests

In [118]:
filteringTest1 :: Property
filteringTest1 = once $
  filtering (ExactlyOne . even) (4 :. 5 :. 6 :. Nil) === ExactlyOne (4 :. 6 :. Nil)

filteringTest2 :: Property
filteringTest2 = once $
  filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. Nil) === Full (4 :. 5 :. 6 :. Nil)

filteringTest3 :: Property
filteringTest3 =
  let
    lhs :: Optional (List Int)
    lhs = filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 7 :. 8 :. 9 :. Nil)
    rhs = Full (4 :. 5 :. 6 :. 7 :. Nil)
  in once $ lhs === rhs

filteringTest4 :: Property
filteringTest4 =
  let
    lhs = filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 13 :. 14 :. Nil)
    rhs = Empty
  in once $ lhs === rhs

filteringTest5 :: Property
filteringTest5 =
  let
    lhs = filtering (const $ True :. True :.  Nil) (1 :. 2 :. 3 :. Nil)
    rhs = replicate 8 (1 :. 2 :. 3 :. Nil)
  in once $ lhs === rhs

runFilteringTests :: IO ()
runFilteringTests = runTests [
    ("filtering (ExactlyOne . even) (4 :. 5 :. 6 :. Nil) === ExactlyOne (4 :. 6 :. Nil)", filteringTest1)
  , ("filtering (\\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. Nil) === Full (4 :. 5 :. 6 :. Nil)", filteringTest2)
  , ("filtering (\\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 7 :. 8 :. 9 :. Nil) === Full (4 :. 5 :. 6 :. Nil)", filteringTest3)
  , ("filtering (\\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 13 :. 14 :. Nil) === Empty", filteringTest4)
  , ("foo", filteringTest5)
  ]

runFilteringTests

Testing: filtering (ExactlyOne . even) (4 :. 5 :. 6 :. Nil) === ExactlyOne (4 :. 6 :. Nil)
+++ OK, passed 1 tests.
Testing: filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. Nil) === Full (4 :. 5 :. 6 :. Nil)
+++ OK, passed 1 tests.
Testing: filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 7 :. 8 :. 9 :. Nil) === Full (4 :. 5 :. 6 :. Nil)
+++ OK, passed 1 tests.
Testing: filtering (\a -> if a > 13 then Empty else Full (a <= 7)) (4 :. 5 :. 6 :. 13 :. 14 :. Nil) === Empty
+++ OK, passed 1 tests.
Testing: foo
+++ OK, passed 1 tests.

### 11. (Advanced) `Applicative` Instance for "Reader"

Implement the `Applicative` Instance for the `Reader r` or `(->) r` `Functor` that was discussed in the `Functor` set of exercises.

#### Exercise

In [121]:
instance Applicative ((->) r) where
  pure :: a -> ((->) r) a
  pure = _pure
  
  (<*>) :: ((->) r) (a -> b) -> ((->) r) a -> ((->) r) b
  (<*>) = _apply

#### Try it out

In [122]:
-- >>> ((+) <*> (+10)) 3
-- 16
((+) <*> (+10)) 3

16

In [123]:
-- >>> ((+) <*> (+5)) 3
-- 11
((+) <*> (+5)) 3

11

In [124]:
-- >>> ((+) <*> (+5)) 1
-- 7
((+) <*> (+5)) 1

7

In [125]:
-- >>> ((*) <*> (+10)) 3
-- 39
((*) <*> (+10)) 3

39

In [128]:
-- >>> ((*) <*> (+2)) 3
-- 15
((*) <*> (+2)) 3

15

#### Tests

In [145]:
functionTest1 :: Property
functionTest1 = forAll arbitrary $ \n ->
  ((+) <*> (+10)) n === (n + (n + 10))

functionTest2 :: Property
functionTest2 = forAll arbitrary $ \n ->
  ((+) <*> (+5)) n === (n + (n + 5))

functionTest3 :: Property
functionTest3 = forAll arbitrary $ \n ->
  ((*) <*> (+10)) n === (n * (n + 10))
  
functionTest4 :: Property
functionTest4 = forAll arbitrary $ \n ->
  ((*) <*> (+2)) n === (n * (n + 2))

functionPure :: Property
functionPure =
  let
    gen :: Gen (Int, Int)
    gen = do
      x <- arbitrary
      y <- arbitrary
      P.return (x, y)
  in forAll gen $ \(x, y) ->
    pure x y === x

functionLift2 :: Property
functionLift2 =
  let
    genN :: Int -> Gen (List Int)
    genN 0 = P.return Nil
    genN n = do
      h <- arbitrary
      t <- genN (n - 1)
      P.return (h :. t)
    
    lists :: Gen (List Int)
    lists = choose (0, 40) P.>>= genN
  in forAll lists $ \xs ->
    lift2 (+) length sum xs === (length xs) + (sum xs)
 
runFunctionTests :: IO ()
runFunctionTests = runTests [
    ("((+) <*> (+10)) n === (n + (n + 10))", functionTest1)
  , ("((+) <*> (+5)) n === (n + (n + 5))", functionTest2)
  , ("((*) <*> (+10)) n === (n * (n + 10))", functionTest3)
  , ("((*) <*> (+2)) n === (n * (n + 2))", functionTest4)
  , ("pure x y === x", functionPure)
  , ("lift2 (+) length sum xs === (length xs) + (sum xs)", functionLift2)
  ]

runFunctionTests

Testing: ((+) <*> (+10)) n === (n + (n + 10))
+++ OK, passed 100 tests.
Testing: ((+) <*> (+5)) n === (n + (n + 5))
+++ OK, passed 100 tests.
Testing: ((*) <*> (+10)) n === (n * (n + 10))
+++ OK, passed 100 tests.
Testing: ((*) <*> (+2)) n === (n * (n + 2))
+++ OK, passed 100 tests.
Testing: pure x y === x
+++ OK, passed 100 tests.
Testing: lift2 (+) length sum xs === (length xs) + (sum xs)
+++ OK, passed 100 tests.

# TODO: There are still more exercises to port over

## Discussion

Things to discuss:

- The point of Applicative.
    - The limitations of `Functor`.
- Monoid and Applicative.
- `lift*`, `<$>`, `<*>` and currying.

But look at the type for `(P.++ "bar")`: